Connected to ragvenv (Python 3.9.6)

In [1]:
import dspy
from dotenv import load_dotenv

import chromadb
from chromadb.utils import embedding_functions
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM

load_dotenv()  # Load environment variables

turbo = dspy.OpenAI(model='gpt-3.5-turbo')

retriever_model = ChromadbRM(
    collection_name='tesla', 
    persist_directory="./teslasec",
    embedding_function=embedding_functions.DefaultEmbeddingFunction(),
    k=5
)

dspy.settings.configure(lm=turbo, rm=retriever_model)

/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dspy.datasets import HotPotQA

In [3]:
import pandas as pd
df=pd.read_csv('../data/groundtruth_eval_dataset_300_titles.csv')
# df.columns =['Unnamed: 0', 'question', 'context', 'answer', 'metadata']
df =df[['question', 'context', 'answer']]

In [4]:
def df2Dataset(df):
    dataset = []

    for question, context, answer in df.values:
        dataset.append(dspy.Example(question=question, context=context, answer=answer).with_inputs('context', "question"))
    return dataset

In [5]:
trainset=df2Dataset(df.iloc[:10])
devset=df2Dataset(df.iloc[11:21])

In [6]:
trainset

[Example({'question': "What is the trading symbol for Tesla's common stock and on which exchange is it registered?", 'context': "page_content='united states securities and exchange commission washington, d. c. 20549 form 10 - k ( mark one ) x annual report pursuant to section 13 or 15 ( d ) of the securities exchange act of 1934 for the fiscal year ended december 31, 2023 or o transition report pursuant to section 13 or 15 ( d ) of the securities exchange act of 1934 for the transition period from _ _ _ _ _ _ _ _ _ to _ _ _ _ _ _ _ _ _ commission file number : 001 - 34756 tesla, inc. ( exact name of registrant as specified in its charter ) delaware 91 - 2197729 ( state or other jurisdiction of incorporation or organization ) ( i. r. s. employer identification no. ) 1 tesla road austin, texas 78725 ( address of principal executive offices ) ( zip code ) ( 512 ) 516 - 8177 ( registrant ’ s telephone number, including area code ) securities registered pursuant to section 12 ( b ) of the a

In [7]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [8]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [9]:
from dsp.utils import deduplicate
class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question, context=None):
        if context is None:
            context = []  # Ensure context is a list if not provided
        
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            retrieved_data = self.retrieve(query)
            passages = retrieved_data.passages if hasattr(retrieved_data, 'passages') else []

            # Ensure both context and passages are lists before concatenation
            if not isinstance(context, list):
                context = [context]
            if not isinstance(passages, list):
                passages = [passages]

            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [10]:
# Ask any question you like to this simple RAG program.
my_question = "what's revenue of tesla 2023?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: what's revenue of tesla 2023?
Predicted Answer: $96,773 million
Retrieved Contexts (truncated): ['other comprehensive loss ( 143 ) ( 361 ) Retained earnings 27,882 12,885 Total stockholders equity 62,634 44,704 Noncontrolling interests in subsidiaries 733 785 Total liabilities and equity $ 106,618...', 'taxes owed, if any. Tesla periodically does business with certain entities with which its CEO and directors are affiliated, such as SpaceX and X Corp., in accordance with our Related Person Transactio...', 'the year ended December 31, 2023 as compared to the year ended December 31, 2022, primarily due to the release of the valuation allowance regarding our U.S. federal and certain state deferred tax asse...', 'used vehicle revenue driven by increases in volume, body shop and part sales revenue, non-warranty maintenance services revenue, paid Supercharging revenue and insurance services revenue, all of which...', 'Gigafactory Documents, dated April 5, 2016, by and among the Reg

In [11]:
turbo.inspect_history(n=3)





Write a simple search query that will help answer a complex question.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context:
[1] «other comprehensive loss ( 143 ) ( 361 ) Retained earnings 27,882 12,885 Total stockholders equity 62,634 44,704 Noncontrolling interests in subsidiaries 733 785 Total liabilities and equity $ 106,618 $ 82,338 The accompanying notes are an integral part of these consolidated financial statements. 49 Tesla, Inc. Consolidated Statements of Operations (in millions, except per share data) Year Ended December 31, 2023 2022 2021 Revenues Automotive sales $ 78,509 $ 67,210 $ 44,125 Automotive regulatory credits 1,790 1,776 1,465 Automotive leasing 2,120 2,476 1,642 Total automotive revenues 82,419 71,462 47,232 Energy generation and storage 6,035 3,909 2,789 Services and other 8,319 6,091 3,802 Total revenues 96,7

In [13]:
class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

metricLM = dspy.OpenAI(model='gpt-4', max_tokens=1000, model_type='chat')

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer  # Assuming `pred` is the prediction object and has an `answer` attribute
    question = gold.question

    print(f"Test Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")

    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"

    with dspy.context(lm=metricLM):
        context = gold.context
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)

    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)

    return total / 5.0

In [14]:
from dspy.teleprompt import BootstrapFewShot

# teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
teleprompter = BootstrapFewShot(metric=llm_metric)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 40%|████      | 4/10 [00:00<00:00, 25.11it/s]

Test Question: What is the trading symbol for Tesla's common stock and on which exchange is it registered?
Predicted Answer: TSLA, The Nasdaq Global Select Market
Faithful: 5
Detail: 2
Overall: 5
Test Question: According to the context, has the registrant filed all reports required by section 13 or 15 (d) of the Securities Exchange Act of 1934 during the preceding 12 months?
Predicted Answer: Yes
Faithful: 5
Detail: 1
Overall: 5
Test Question: According to the given context, is the registrant a large accelerated filer?
Predicted Answer: Yes
Faithful: 5
Detail: 1
Overall: 5
Test Question: According to the given context, has the registrant elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to section 13(a) of the Exchange Act?
Predicted Answer: Yes
Faithful: 5
Detail: 1
Overall: 1
Bootstrapped 4 full traces after 5 examples in round 0.


In [15]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

In [16]:
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=llm_metric)

Average Metric: 17.2 / 6  (286.7):  50%|█████     | 5/10 [00:00<00:00, 23.04it/s]

Test Question: What are some strategies Tesla uses to accelerate the widespread adoption of their products?
Predicted Answer: Offer financial and other services
Faithful: 5
Detail: 2
Overall: 3
Test Question: What are the two reportable segments of the company?
Predicted Answer: automotive and energy generation and storage
Faithful: 5
Detail: 2
Overall: 5
Test Question: What are the five different consumer vehicles currently manufactured by the company?
Predicted Answer: Model 3, Y, S, X, Cybertruck
Faithful: 5
Detail: 2
Overall: 5
Test Question: What is the purpose of the Tesla Semi?
Predicted Answer: address additional vehicle markets
Faithful: 5
Detail: 1
Overall: 5
Test Question: What are some markets and applications that can be accessed through our real-time energy control and optimization platforms?
Predicted Answer: energy generation, storage
Faithful: 1
Detail: 1
Overall: 1
Test Question: What are the different ways in which the company sells its retrofit solar energy systems?

Average Metric: 20.4 / 7  (291.4):  60%|██████    | 6/10 [00:00<00:00, 23.04it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Test Question: What are the unique advantages of our electric powertrain systems?
Predicted Answer: adaptable, efficient, reliable, cost-effective
Faithful: 5
Detail: 1
Overall: 5
Test Question: What is the purpose of the control software in our vehicles?
Predicted Answer: optimize performance, customize behavior, manage charging, control infotainment.


Average Metric: 29.599999999999998 / 10  (296.0): 100%|██████████| 10/10 [00:14<00:00,  1.47s/it]

Faithful: 5
Detail: 2
Overall: 5
Test Question: What functionality do our advanced driver assist systems provide in our vehicles?
Predicted Answer: safety and convenience functionality
Faithful: 5
Detail: 1
Overall: 5
Test Question: How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?
Predicted Answer: By taking a modular approach.
Faithful: 5
Detail: 1
Overall: 2
Average Metric: 29.599999999999998 / 10  (296.0%)



/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['3.0' '3.4' '3.4' '3.2' '0.8']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_context,example_answer,pred_context,pred_answer,llm_metric
0,What are some strategies Tesla uses to accelerate the widespread adoption of their products?,"page_content='table of contents part i item 1. business overview we design, develop, manufacture, sell and lease high - performance fully electric vehicles and energy generation...","Tesla employs several strategies to accelerate the widespread adoption of their products. These include selling their products directly to customers, expanding their customer-facing infrastructure through...","[""page_content='table of contents part i item 1. business overview we design, develop, manufacture, sell and lease high - performance fully electric vehicles and energy generation...",Offer financial and other services,3.0
1,What are the two reportable segments of the company?,"page_content='our mission is to accelerate the world ’ s transition to sustainable energy. we believe that this mission, along with our engineering expertise, vertically integrated...",The two reportable segments of the company are (i) automotive and (ii) energy generation and storage.,"[""page_content='our mission is to accelerate the world ’ s transition to sustainable energy. we believe that this mission, along with our engineering expertise, vertically integrated...",automotive and energy generation and storage,3.4
2,What are the five different consumer vehicles currently manufactured by the company?,"page_content='services and sales of solar energy systems incentives. our products and services automotive we currently manufacture five different consumer vehicles – the model 3, y,...","The five different consumer vehicles currently manufactured by the company are the Model 3, Model Y, Model S, Model X, and Cybertruck.","[""page_content='services and sales of solar energy systems incentives. our products and services automotive we currently manufacture five different consumer vehicles – the model 3, y,...","Model 3, Y, S, X, Cybertruck",3.4
3,What is the purpose of the Tesla Semi?,"page_content='in 2022, we also began early production and deliveries of a commercial electric vehicle, the tesla semi. we have planned electric vehicles to address additional...","The purpose of the Tesla Semi is to serve as a commercial electric vehicle, addressing additional vehicle markets and leveraging Tesla's developments in full self-driving...","[""page_content='in 2022, we also began early production and deliveries of a commercial electric vehicle, the tesla semi. we have planned electric vehicles to address additional...",address additional vehicle markets,3.2
4,What are some markets and applications that can be accessed through our real-time energy control and optimization platforms?,"page_content='markets and applications, including through our real - time energy control and optimization platforms. 4' metadata={'page_number': 5}","The real-time energy control and optimization platforms can access various markets and applications. These include energy management for industrial and commercial buildings, integration with renewable...","[""page_content='markets and applications, including through our real - time energy control and optimization platforms. 4' metadata={'page_number': 5}"", 'loss up to a certain limit or repurchase...","energy generation, storage",0.8


In [17]:
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=llm_metric)

print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

Average Metric: 19.4 / 6  (323.3):  60%|██████    | 6/10 [00:00<00:00, 25.86it/s]              

Test Question: What are some strategies Tesla uses to accelerate the widespread adoption of their products?
Predicted Answer: Tesla emphasizes performance, attractive styling, safety, and continuous efforts to reduce manufacturing costs. They also offer financial and other services tailored to their products.
Faithful: 5
Detail: 5
Overall: 5
Test Question: What are the two reportable segments of the company?
Predicted Answer: Automotive and Energy Generation and Storage
Faithful: 5
Detail: 1
Overall: 5
Test Question: What are the five different consumer vehicles currently manufactured by the company?
Predicted Answer: Model 3, Y, S, X, Cybertruck
Faithful: 5
Detail: 2
Overall: 5
Test Question: What is the purpose of the Tesla Semi?
Predicted Answer: Commercial electric vehicle
Faithful: 5
Detail: 1
Overall: 3
Test Question: What are some markets and applications that can be accessed through our real-time energy control and optimization platforms?
Predicted Answer: Markets and applicati

Average Metric: 33.199999999999996 / 10  (332.0): 100%|██████████| 10/10 [00:00<00:00, 25.72it/s]

Test Question: What are the unique advantages of our electric powertrain systems?
Predicted Answer: Adaptable, efficient, reliable, cost-effective, withstanding automotive rigors.
Faithful: 5
Detail: 2
Overall: 5
Test Question: What is the purpose of the control software in our vehicles?
Predicted Answer: Optimize performance, customize behavior, manage charging, control infotainment functions.
Faithful: 5
Detail: 2
Overall: 5
Test Question: What functionality do our advanced driver assist systems provide in our vehicles?
Predicted Answer: Safety and convenience functionality
Faithful: 5
Detail: 2
Overall: 5
Test Question: How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?
Predicted Answer: By leveraging component-level technologies from vehicles and designing battery systems modularly, we can optimize manufacturing capacity for energy storage products.
Faithful: 5
Detail: 3
Overall: 5
Average Metric: 33.199


/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4.0' '3.2' '3.4' '2.8' '2.4']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_context,example_answer,pred_context,pred_answer,llm_metric
0,What are some strategies Tesla uses to accelerate the widespread adoption of their products?,"page_content='table of contents part i item 1. business overview we design, develop, manufacture, sell and lease high - performance fully electric vehicles and energy generation...","Tesla employs several strategies to accelerate the widespread adoption of their products. These include selling their products directly to customers, expanding their customer-facing infrastructure through...","[""page_content='table of contents part i item 1. business overview we design, develop, manufacture, sell and lease high - performance fully electric vehicles and energy generation...","Tesla emphasizes performance, attractive styling, safety, and continuous efforts to reduce manufacturing costs. They also offer financial and other services tailored to their products.",4.0
1,What are the two reportable segments of the company?,"page_content='our mission is to accelerate the world ’ s transition to sustainable energy. we believe that this mission, along with our engineering expertise, vertically integrated...",The two reportable segments of the company are (i) automotive and (ii) energy generation and storage.,"[""page_content='our mission is to accelerate the world ’ s transition to sustainable energy. we believe that this mission, along with our engineering expertise, vertically integrated...",Automotive and Energy Generation and Storage,3.2
2,What are the five different consumer vehicles currently manufactured by the company?,"page_content='services and sales of solar energy systems incentives. our products and services automotive we currently manufacture five different consumer vehicles – the model 3, y,...","The five different consumer vehicles currently manufactured by the company are the Model 3, Model Y, Model S, Model X, and Cybertruck.","[""page_content='services and sales of solar energy systems incentives. our products and services automotive we currently manufacture five different consumer vehicles – the model 3, y,...","Model 3, Y, S, X, Cybertruck",3.4
3,What is the purpose of the Tesla Semi?,"page_content='in 2022, we also began early production and deliveries of a commercial electric vehicle, the tesla semi. we have planned electric vehicles to address additional...","The purpose of the Tesla Semi is to serve as a commercial electric vehicle, addressing additional vehicle markets and leveraging Tesla's developments in full self-driving...","[""page_content='in 2022, we also began early production and deliveries of a commercial electric vehicle, the tesla semi. we have planned electric vehicles to address additional...",Commercial electric vehicle,2.8
4,What are some markets and applications that can be accessed through our real-time energy control and optimization platforms?,"page_content='markets and applications, including through our real - time energy control and optimization platforms. 4' metadata={'page_number': 5}","The real-time energy control and optimization platforms can access various markets and applications. These include energy management for industrial and commercial buildings, integration with renewable...","[""page_content='markets and applications, including through our real - time energy control and optimization platforms. 4' metadata={'page_number': 5}"", 'loss up to a certain limit or repurchase...",Markets and applications,2.4


## Retrieval Score for uncompiled Baleen: 296.0
## Retrieval Score for compiled Baleen: 332.0


In [18]:
import dspy
from dotenv import load_dotenv

import chromadb
from chromadb.utils import embedding_functions
import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM

load_dotenv()  # Load environment variables

turbo = dspy.OpenAI(model='gpt-3.5-turbo')

retriever_model = ChromadbRM(
    collection_name='tesla', 
    persist_directory="./teslasec",
    embedding_function=embedding_functions.DefaultEmbeddingFunction(),
    k=5
)

dspy.settings.configure(lm=turbo, rm=retriever_model)

In [19]:
from dspy.datasets import HotPotQA

In [20]:
import pandas as pd
df=pd.read_csv('../data/groundtruth_eval_dataset_300_titles.csv')
# df.columns =['Unnamed: 0', 'question', 'context', 'answer', 'metadata']
df =df[['question', 'context', 'answer']]

In [21]:
def df2Dataset(df):
    dataset = []

    for question, context, answer in df.values:
        dataset.append(dspy.Example(question=question, context=context, answer=answer).with_inputs('context', "question"))
    return dataset

In [22]:
trainset=df2Dataset(df.iloc[:20])
devset=df2Dataset(df.iloc[20:41])

In [23]:
trainset

[Example({'question': "What is the trading symbol for Tesla's common stock and on which exchange is it registered?", 'context': "page_content='united states securities and exchange commission washington, d. c. 20549 form 10 - k ( mark one ) x annual report pursuant to section 13 or 15 ( d ) of the securities exchange act of 1934 for the fiscal year ended december 31, 2023 or o transition report pursuant to section 13 or 15 ( d ) of the securities exchange act of 1934 for the transition period from _ _ _ _ _ _ _ _ _ to _ _ _ _ _ _ _ _ _ commission file number : 001 - 34756 tesla, inc. ( exact name of registrant as specified in its charter ) delaware 91 - 2197729 ( state or other jurisdiction of incorporation or organization ) ( i. r. s. employer identification no. ) 1 tesla road austin, texas 78725 ( address of principal executive offices ) ( zip code ) ( 512 ) 516 - 8177 ( registrant ’ s telephone number, including area code ) securities registered pursuant to section 12 ( b ) of the a

In [24]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [25]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

In [26]:
from dsp.utils import deduplicate
class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question, context=None):
        if context is None:
            context = []  # Ensure context is a list if not provided
        
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            retrieved_data = self.retrieve(query)
            passages = retrieved_data.passages if hasattr(retrieved_data, 'passages') else []

            # Ensure both context and passages are lists before concatenation
            if not isinstance(context, list):
                context = [context]
            if not isinstance(passages, list):
                passages = [passages]

            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [27]:
# Ask any question you like to this simple RAG program.
my_question = "what's revenue of tesla 2022?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: what's revenue of tesla 2022?
Predicted Answer: $81.462 billion
Retrieved Contexts (truncated): ['333-187113, 333-183033, and 333-167874) of Tesla, Inc. of our report dated January 26, 2024 relating to the financial statements and the effectiveness of internal control over financial reporting, whi...', 'on our net income before income taxes. These changes would have resulted in a gain or loss of $1.01 billion at December 31, 2023 and $473 million at December 31, 2022, assuming no foreign currency hed...', 'other comprehensive loss ( 143 ) ( 361 ) Retained earnings 27,882 12,885 Total stockholders equity 62,634 44,704 Noncontrolling interests in subsidiaries 733 785 Total liabilities and equity $ 106,618...', 'used vehicle revenue driven by increases in volume, body shop and part sales revenue, non-warranty maintenance services revenue, paid Supercharging revenue and insurance services revenue, all of which...', 'service and pricing, educate consumers about electric vehicles, 

In [28]:
turbo.inspect_history(n=3)





Write a simple search query that will help answer a complex question.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context:
[1] «333-187113, 333-183033, and 333-167874) of Tesla, Inc. of our report dated January 26, 2024 relating to the financial statements and the effectiveness of internal control over financial reporting, which appears in this Form 10-K. /s/ PricewaterhouseCoopers LLP San Jose, California January 26, 2024 Exhibit 31.1 CERTIFICATIONS I, Elon Musk, certify that: 1. I have reviewed this Annual Report on Form 10-K of Tesla, Inc.; 2. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a material fact necessary to make the statements made, in light of the circumstances under which such statements were made, not misleading with respect to the period covered by this 

In [30]:
class Assess(dspy.Signature):
    """Assess the quality of an answer to a question."""
    
    context = dspy.InputField(desc="The context for answering the question.")
    assessed_question = dspy.InputField(desc="The evaluation criterion.")
    assessed_answer = dspy.InputField(desc="The answer to the question.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

metricLM = dspy.OpenAI(model='gpt-4', max_tokens=1000, model_type='chat')

def llm_metric(gold, pred, trace=None):
    predicted_answer = pred.answer  # Assuming `pred` is the prediction object and has an `answer` attribute
    question = gold.question

    print(f"Test Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")

    detail = "Is the assessed answer detailed?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    overall = f"Please rate how well this answer answers the question, `{question}` based on the context.\n `{predicted_answer}`"

    with dspy.context(lm=metricLM):
        context = gold.context
        detail = dspy.ChainOfThought(Assess)(context="N/A", assessed_question=detail, assessed_answer=predicted_answer)
        faithful = dspy.ChainOfThought(Assess)(context=context, assessed_question=faithful, assessed_answer=predicted_answer)
        overall = dspy.ChainOfThought(Assess)(context=context, assessed_question=overall, assessed_answer=predicted_answer)

    print(f"Faithful: {faithful.assessment_answer}")
    print(f"Detail: {detail.assessment_answer}")
    print(f"Overall: {overall.assessment_answer}")
    total = float(detail.assessment_answer) + float(faithful.assessment_answer)*2 + float(overall.assessment_answer)

    return total / 5.0

In [31]:
from dspy.teleprompt import BootstrapFewShot

# teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
teleprompter = BootstrapFewShot(metric=llm_metric)
compiled_baleen = teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)

  5%|▌         | 1/20 [00:01<00:23,  1.21s/it]

Test Question: What is the trading symbol for Tesla's common stock and on which exchange is it registered?
Predicted Answer: TSLA, The Nasdaq Global Select Market
Faithful: 5
Detail: 2
Overall: 5


 10%|█         | 2/20 [00:02<00:25,  1.40s/it]

Test Question: According to the context, has the registrant filed all reports required by section 13 or 15 (d) of the Securities Exchange Act of 1934 during the preceding 12 months?
Predicted Answer: Yes
Faithful: 5
Detail: 1
Overall: 5


 15%|█▌        | 3/20 [00:03<00:20,  1.23s/it]

Test Question: According to the given context, is the registrant a large accelerated filer?
Predicted Answer: Yes
Faithful: 5
Detail: 1
Overall: 5
Test Question: According to the given context, has the registrant elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to section 13(a) of the Exchange Act?
Predicted Answer: No


 20%|██        | 4/20 [00:16<01:04,  4.02s/it]

Faithful: 5
Detail: 1
Overall: 5
Bootstrapped 4 full traces after 5 examples in round 0.


In [32]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

In [33]:
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=llm_metric)

Average Metric: 2.6 / 1  (260.0):   0%|          | 0/21 [00:00<?, ?it/s]

Test Question: How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?
Predicted Answer: By taking a modular approach.
Faithful: 5
Detail: 1
Overall: 2
Test Question: What are some specific engineering feats achieved by our team in the design and test engineering of electric vehicles and their components and systems?
Predicted Answer: integrating batteries directly with vehicle body structures


Average Metric: 5.800000000000001 / 2  (290.0):  10%|▉         | 2/21 [00:16<02:35,  8.17s/it]

Faithful: 5
Detail: 1
Overall: 5
Test Question: What expertise allows us to design and engineer energy generating and storage products?
Predicted Answer: electrical, mechanical, civil, and software engineering


Average Metric: 9.200000000000001 / 3  (306.7):  14%|█▍        | 3/21 [00:33<03:30, 11.72s/it]

...

Faithful: 5
Detail: 2
Overall: 5
Test Question: What is the role of the National Highway Traffic Safety Administration (NHTSA) in regulating vehicles in the United States?
Predicted Answer: Regulating vehicle safety and compliance.


Average Metric: 61.39999999999999 / 21  (292.4): 100%|██████████| 21/21 [05:06<00:00, 14.58s/it]

Faithful: 5
Detail: 2
Overall: 5
Average Metric: 61.39999999999999 / 21  (292.4%)



/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2.6' '3.2' '3.4' '2.4' '3.6']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_context,example_answer,pred_context,pred_answer,llm_metric,context,answer
0,How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?,page_content='table of contents energy generation and storage energy storage products we leverage many of the component - level technologies from our vehicles in our energy...,"To optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design, we leverage many of the component-level technologies...","[""page_content='table of contents energy generation and storage energy storage products we leverage many of the component - level technologies from our vehicles in our energy...",By taking a modular approach.,2.6,nan,nan
1,What are some specific engineering feats achieved by our team in the design and test engineering of electric vehicles and their components and systems?,page_content='we have established significant in - house capabilities in the design and test engineering of electric vehicles and their components and systems. our team has...,Our team has achieved several specific engineering feats in the design and test engineering of electric vehicles and their components and systems. These include significant...,"[""page_content='we have established significant in - house capabilities in the design and test engineering of electric vehicles and their components and systems. our team has...",integrating batteries directly with vehicle body structures,3.2,nan,nan
2,What expertise allows us to design and engineer energy generating and storage products?,"page_content='areas of automation, die - making and line - building, we are also making strides in the simulations modeling these capabilities prior to construction. energy...","Our expertise in electrical, mechanical, civil, and software engineering allows us to design, engineer, manufacture, and install energy generating and storage products and components.","[""page_content='areas of automation, die - making and line - building, we are also making strides in the simulations modeling these capabilities prior to construction. energy...","electrical, mechanical, civil, and software engineering",3.4,nan,nan
3,How does Tesla tailor their marketing efforts?,"page_content='public narrative and brand, and tailor our marketing efforts accordingly, including through investments in customer education and advertising as necessary. automotive direct sales our vehicle...",Tesla tailors their marketing efforts by investing in customer education and advertising as necessary. They utilize a combination of direct sales through their website and...,"[""page_content='public narrative and brand, and tailor our marketing efforts accordingly, including through investments in customer education and advertising as necessary. automotive direct sales our vehicle...",monitor public narrative and brand,2.4,nan,nan
4,How does our used vehicle business support new vehicle sales?,page_content='table of contents used vehicle sales our used vehicle business supports new vehicle sales by integrating the trade - in of a customer ’ s...,Our used vehicle business supports new vehicle sales by integrating the trade-in of a customer's existing Tesla or non-Tesla vehicle with the sale of a...,"[""page_content='table of contents used vehicle sales our used vehicle business supports new vehicle sales by integrating the trade - in of a customer ’ s...",by integrating the trade-in of a customer's existing Tesla or non-Tesla vehicle with the sale of a new or used Tesla vehicle.,3.6,nan,nan


In [34]:
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, metric=llm_metric)

print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

  0%|          | 0/21 [00:00<?, ?it/s]

Test Question: How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?
Predicted Answer: Tesla optimizes manufacturing capacity by taking a modular approach to battery system design, allowing them to leverage component-level technologies from their vehicles.


Average Metric: 4.0 / 1  (400.0):   5%|▍         | 1/21 [00:13<04:29, 13.46s/it]

Faithful: 5
Detail: 5
Overall: 5
Test Question: What are some specific engineering feats achieved by our team in the design and test engineering of electric vehicles and their components and systems?
Predicted Answer: Achieved feats in stamping, casting, thermal systems, and battery integration.


Average Metric: 7.4 / 2  (370.0):  10%|▉         | 2/21 [00:28<04:32, 14.34s/it]

...

Average Metric: 76.0 / 21  (361.9): 100%|██████████| 21/21 [09:43<00:00, 27.79s/it]

Faithful: 5
Detail: 3
Overall: 5
Average Metric: 76.0 / 21  (361.9%)



/Users/hongtang/Documents/RAG_Implementation/ragvenv/lib/python3.9/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4.0' '3.4' '3.4' '4.0' '3.4']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_context,example_answer,pred_context,pred_answer,llm_metric
0,How do we optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design?,page_content='table of contents energy generation and storage energy storage products we leverage many of the component - level technologies from our vehicles in our energy...,"To optimize the manufacturing capacity of our energy storage products through a modular approach to battery system design, we leverage many of the component-level technologies...","[""page_content='table of contents energy generation and storage energy storage products we leverage many of the component - level technologies from our vehicles in our energy...","Tesla optimizes manufacturing capacity by taking a modular approach to battery system design, allowing them to leverage component-level technologies from their vehicles.",4.0
1,What are some specific engineering feats achieved by our team in the design and test engineering of electric vehicles and their components and systems?,page_content='we have established significant in - house capabilities in the design and test engineering of electric vehicles and their components and systems. our team has...,Our team has achieved several specific engineering feats in the design and test engineering of electric vehicles and their components and systems. These include significant...,"[""page_content='we have established significant in - house capabilities in the design and test engineering of electric vehicles and their components and systems. our team has...","Achieved feats in stamping, casting, thermal systems, and battery integration.",3.4
2,What expertise allows us to design and engineer energy generating and storage products?,"page_content='areas of automation, die - making and line - building, we are also making strides in the simulations modeling these capabilities prior to construction. energy...","Our expertise in electrical, mechanical, civil, and software engineering allows us to design, engineer, manufacture, and install energy generating and storage products and components.","[""page_content='areas of automation, die - making and line - building, we are also making strides in the simulations modeling these capabilities prior to construction. energy...","Electrical, mechanical, civil, and software engineering expertise",3.4
3,How does Tesla tailor their marketing efforts?,"page_content='public narrative and brand, and tailor our marketing efforts accordingly, including through investments in customer education and advertising as necessary. automotive direct sales our vehicle...",Tesla tailors their marketing efforts by investing in customer education and advertising as necessary. They utilize a combination of direct sales through their website and...,"[""page_content='public narrative and brand, and tailor our marketing efforts accordingly, including through investments in customer education and advertising as necessary. automotive direct sales our vehicle...",Tesla tailors their marketing efforts by investing in customer education and advertising as necessary to maintain public credibility and confidence in their long-term business prospects.,4.0
4,How does our used vehicle business support new vehicle sales?,page_content='table of contents used vehicle sales our used vehicle business supports new vehicle sales by integrating the trade - in of a customer ’ s...,Our used vehicle business supports new vehicle sales by integrating the trade-in of a customer's existing Tesla or non-Tesla vehicle with the sale of a...,"[""page_content='table of contents used vehicle sales our used vehicle business supports new vehicle sales by integrating the trade - in of a customer ’ s...",By integrating the trade-in of a customer's existing Tesla or non-Tesla vehicle with the sale of a new or used Tesla vehicle.,3.4


## Retrieval Score for uncompiled Baleen: 292.38
## Retrieval Score for compiled Baleen: 361.9
